In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import RandomizedSearchCV, KFold
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error

-
train = pd.read_csv("Train.csv")
test = pd.read_csv("Test.csv")
submission = pd.read_csv("Submission.csv")


for col in train.columns:
    if train[col].isna().sum() > 0:
        train[col].fillna(train[col].median(), inplace=True)

for col in test.columns:
    if test[col].isna().sum() > 0:
        test[col].fillna(train[col].median(), inplace=True)


X_train = train.drop('protein_mass', axis=1)
y_train = train['protein_mass']
X_test = test.drop('protein_mass', axis=1)


def tune_lgbm(X, y):
    lgb = LGBMRegressor(random_state=42)
    param_dist = {
        'num_leaves': [31, 50, 70],
        'learning_rate': [0.05, 0.1, 0.2],
        'n_estimators': [300, 500, 800],
        'max_depth': [-1, 5, 10, 15]
    }
    rs = RandomizedSearchCV(
        lgb,
        param_dist,
        cv=5,
        scoring='neg_root_mean_squared_error',
        n_iter=20,
        random_state=42,
        n_jobs=-1
    )
    rs.fit(X, y)
    print("Best LightGBM params:", rs.best_params_)
    return rs.best_estimator_


def tune_catboost(X, y):
    cb = CatBoostRegressor(random_state=42, verbose=0)
    param_dist = {
        'depth': [4, 6, 8, 10],
        'learning_rate': [0.01, 0.05, 0.1],
        'iterations': [300, 500, 800],
        'l2_leaf_reg': [1, 3, 5, 7]
    }
    rs = RandomizedSearchCV(
        cb,
        param_dist,
        cv=5,
        scoring='neg_root_mean_squared_error',
        n_iter=20,
        random_state=42,
        n_jobs=-1
    )
    rs.fit(X, y)
    print("Best CatBoost params:", rs.best_params_)
    return rs.best_estimator_


lgb_model = tune_lgbm(X_train, y_train)
cb_model = tune_catboost(X_train, y_train)


kf = KFold(n_splits=5, shuffle=True, random_state=42)
lgb_cv_scores = []
cb_cv_scores = []


C:\Users\ahpat\AppData\Local\Temp\ipykernel_20084\1488271856.py:29: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  test[col].fillna(train[col].median(), inplace=True)


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.006986 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1530
[LightGBM] [Info] Number of data points in the train set: 14700, number of used features: 6
[LightGBM] [Info] Start training from score 459.128128
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain

TypeError: got an unexpected keyword argument 'squared'

In [4]:

for train_index, val_index in kf.split(X_train):
    X_tr, X_val = X_train.iloc[train_index], X_train.iloc[val_index]
    y_tr, y_val = y_train.iloc[train_index], y_train.iloc[val_index]

    lgb_model.fit(X_tr, y_tr)
    preds_lgb = lgb_model.predict(X_val)
    lgb_cv_scores.append(mean_squared_error(y_val, preds_lgb))

    cb_model.fit(X_tr, y_tr)
    preds_cb = cb_model.predict(X_val)
    cb_cv_scores.append(mean_squared_error(y_val, preds_cb))

print(f"LightGBM CV RMSE: {np.mean(lgb_cv_scores):.4f}")
print(f"CatBoost CV RMSE: {np.mean(cb_cv_scores):.4f}")


lgb_model.fit(X_train, y_train)
cb_model.fit(X_train, y_train)

preds_lgb = lgb_model.predict(X_test)
preds_cb = cb_model.predict(X_test)

final_preds = (preds_lgb + preds_cb) / 2




[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000234 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1530
[LightGBM] [Info] Number of data points in the train set: 11760, number of used features: 6
[LightGBM] [Info] Start training from score 459.094843
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain

In [6]:
submission['protein_mass'] = final_preds
submission.to_csv("final_submission.csv", index=False)

print("✅ Submission file 'final_submission.csv' is ready.")

✅ Submission file 'final_submission.csv' is ready.
